# Create watersheds and catchments

AKSSF project has ~ 500 sites that have been shifted to the flow networks.
We need to create watersheds for each. This will make extracting spatial
and climatic covariates for modeling go much faster.

Dustin pointed out that there are fromnodes and tonodes in the NHDPlus that can be used to navigate
upstream, save all the NHPPlusIDs, select and merge the catchments to create watersheds for each site.
This works in R so just need to transfer to python. Premise is to use a while loop to keep selecting
new stream segments that have their tonode match the fromnode of the last segment(s). Stop when
summing the ids is not greater than 0.

Create a loop and process watersheds for all the points. Start with Cook Inlet first.
Note that folders, geodbs, and merged catchments are created in the merge_grids script.
1. select catchments that intersect points to get NHDPlusID
2. create list of IDs
3. use loop to create watersheds
4. first get list of all upstream NHDPlusIDs
5. create temporary layer of catchments
6. select catchments that match the upstream IDs
7. dissolved on those catchments and save to Cook Inlet gdb and watersheds feature dataset

In [21]:
# steps 1 and 2
# intersect points with catchments and create list of NHDPlusIDs
import arcpy
arcpy.env.workspace = r"W:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb"
arcpy.env.overwriteOutput = True

points = r"T:\Aquatic\AKSSF\AKSSF_Hydrography.gdb\sites_outside_bb_verified_DM"
cats = r"W:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_merge"
idList = []
outcats = "cats_intersect"

arcpy.MakeFeatureLayer_management(cats, "tempLayer")
arcpy.management.SelectLayerByLocation("tempLayer", "INTERSECT", points)
arcpy.management.SaveToLayerFile("tempLayer", outcats)

fields = arcpy.ListFields("tempLayer")
for field in fields:
    print("{0}".format(field.name))
with arcpy.da.SearchCursor("tempLayer", ["NHDPlusID"]) as cursor:
    for row in cursor:
        idList.append(row[0])

print(len(idList))



OBJECTID
Shape
NHDPlusID
SourceFC
GridCode
AreaSqKm
VPUID
Catch_ID
Shape_Length
Shape_Area
241


In [ ]:
import arcpy
import pandas as pd

#idList = [75004300004324]

# steps 4-9 for loop to create watersheds
arcpy.env.workspace = r"W:/GIS/AKSSF/Cook_Inlet/Cook_Inlet.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False

vaa = "vaa_merge"
cats = "cats_merge"
output_SR = arcpy.Describe(cats).spatialReference
arcpy.env.outputCoordinateSystem = output_SR

#watersheds feature dataset for storing fcs
arcpy.management.CreateFeatureDataset(r"W:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb", "Watersheds", output_SR)

vaa_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(vaa, ("NHDPlusID", "FromNode", "ToNode")))

#idList = [75004300006440]

for id in idList:
    print("Starting watershed for: " + str(id))
    rec = [id]
    print(type(rec))
    up_ids = []

    while sum(rec) > 0:
        up_ids.append(rec)
        fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
        rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]

    #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
    newup_ids = []
    for x in up_ids:
        newup_ids.extend(x)

    print(type(newup_ids))
    tempLayer = "catsLyr"
    #expression = 'NHDPlusID IN {0}'.format(tuple(newup_ids))
    #trying expression to deal with one catchment (i.e. hws)
    expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
    arcpy.MakeFeatureLayer_management(cats, tempLayer)
    arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)

    outwtd = "Watersheds\\wtd_" + str(round(id))
    print(outwtd)
    arcpy.management.Dissolve(tempLayer, outwtd)
    #arcpy.management.CopyFeatures(tempLayer, outwtd)

Starting watershed for: 75004200007057.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004200007057
Starting watershed for: 75004300006312.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300006312
Starting watershed for: 75004300001906.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300001906
Starting watershed for: 75004300000100.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300000100
Starting watershed for: 75004300004983.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300004983
Starting watershed for: 75004300004332.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300004332
Starting watershed for: 75004300006239.0
<class 'list'>
<class 'list'>
Watersheds\wtd_75004300006239
Starting watershed for: 75004300004304.0
<class 'list'>
<class 'list'>
